In [1]:
from pyspark.sql import SparkSession

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
14,application_1700859524548_0015,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-14>

In [5]:
df=spark.read.csv("s3://jupyterjorge/raw/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-1K.csv",inferSchema=True,header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#columns of dataframe
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['fecha reporte web', 'ID de caso', 'Fecha de notificación', 'Código DIVIPOLA departamento', 'Nombre departamento', 'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad', 'Unidad de medida de edad', 'Sexo', 'Tipo de contagio', 'Ubicación del caso', 'Estado', 'Código ISO del país', 'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas', 'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación', 'Tipo de recuperación', 'Pertenencia étnica', 'Nombre del grupo étnico']

In [7]:
#check number of columns
len(df.columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

23

In [8]:
#number of records in dataframe
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1000

In [9]:
#shape of dataset
print((df.count(),len(df.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1000, 23)

In [10]:
#printSchema
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de r

In [11]:
#fisrt few rows of dataframe
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+---------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+---------

In [13]:

#select only 2 columns
df.select('Nombre departamento','Nombre municipio').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------+
|Nombre departamento|Nombre municipio|
+-------------------+----------------+
|             BOGOTA|          BOGOTA|
|              VALLE|            BUGA|
|          ANTIOQUIA|        MEDELLIN|
|          ANTIOQUIA|        MEDELLIN|
|          ANTIOQUIA|        MEDELLIN|
+-------------------+----------------+
only showing top 5 rows

In [14]:
#info about dataframe
df.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+-----------------+---------------------+----------------------------+-------------------+-------------------------+----------------+-----------------+------------------------+----+----------------+------------------+---------+-------------------+---------------+----------+---------------------------+----------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|summary|fecha reporte web|       ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|             Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|   Estado|Código ISO del país|Nombre del país|Recuperado|Fecha de inicio de síntomas| Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-------+-----------------+-----------------+---------------------+---------------------------

In [15]:
from pyspark.sql.types import StringType,DoubleType,IntegerType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#with column
df.withColumn("Edad_after_10_yrs",(df["Edad"]+10)).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+-------------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+-----------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país          |Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|Edad_after_10_yrs|
+-----------------+----------+---------------------+----------------------------+-------------------+-

In [17]:
df.withColumn('Edad_double',df['Edad'].cast(DoubleType())).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+-------------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+-----------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país          |Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|Edad_double|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------

In [18]:
#with column
df.withColumn("Edad_after_10_yrs",(df["Edad"]+10)).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+-------------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+-----------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país          |Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|Edad_after_10_yrs|
+-----------------+----------+---------------------+----------------------------+-------------------+-

In [23]:
#filter the records
df.filter(df['Tipo de contagio']=='Importado').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+--------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|     Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----

In [24]:
#filter the records
df.filter(df['Tipo de contagio']=='Importado').select('Edad','Sexo','Tipo de contagio').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+----------------+
|Edad|Sexo|Tipo de contagio|
+----+----+----------------+
|  19|   F|       Importado|
|  34|   M|       Importado|
|  50|   F|       Importado|
|  85|   F|       Importado|
|  22|   F|       Importado|
|  28|   F|       Importado|
|  36|   F|       Importado|
|  42|   F|       Importado|
|  74|   F|       Importado|
|  48|   M|       Importado|
|  30|   F|       Importado|
|  61|   F|       Importado|
|  73|   F|       Importado|
|  54|   M|       Importado|
|  36|   M|       Importado|
|  18|   F|       Importado|
|  49|   M|       Importado|
|  22|   M|       Importado|
|  48|   F|       Importado|
|  65|   M|       Importado|
+----+----+----------------+
only showing top 20 rows

In [25]:
#filter the multiple conditions
df.filter(df['Tipo de contagio']=='Importado').filter(df['ID de caso'] >10).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+--------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|     Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----

In [26]:
#filter the multiple conditions
df.filter((df['Tipo de contagio']=='Importado')&(df['ID de caso'] >10)).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+--------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|     Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----

In [27]:
#Distinct Values in a column
df.select('Tipo de contagio').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|Tipo de contagio|
+----------------+
|       Importado|
|     Comunitaria|
|      En estudio|
|     Relacionado|
+----------------+

In [28]:
#distinct value count
df.select('Tipo de contagio').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4

In [29]:
df.groupBy('Tipo de contagio').count().show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|Tipo de contagio|count|
+----------------+-----+
|Importado       |477  |
|Comunitaria     |63   |
|En estudio      |1    |
|Relacionado     |459  |
+----------------+-----+

In [30]:
# Value counts
df.groupBy('Tipo de contagio').count().orderBy('count',ascending=False).show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|Tipo de contagio|count|
+----------------+-----+
|Importado       |477  |
|Relacionado     |459  |
|Comunitaria     |63   |
|En estudio      |1    |
+----------------+-----+

In [31]:
# Value counts
df.groupBy('Tipo de contagio').mean().show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+---------------------------------+------------------------------+-----------------+-----------------------------+------------------------+-----------------------+
|Tipo de contagio|avg(ID de caso)   |avg(Código DIVIPOLA departamento)|avg(Código DIVIPOLA municipio)|avg(Edad)        |avg(Unidad de medida de edad)|avg(Código ISO del país)|avg(Pertenencia étnica)|
+----------------+------------------+---------------------------------+------------------------------+-----------------+-----------------------------+------------------------+-----------------------+
|Importado       |425.77358490566036|729.1802935010483                |29698.71278825996             |42.44863731656184|1.0                          |640.125786163522        |5.89517819706499       |
|Comunitaria     |776.8730158730159 |637.2380952380952                |18860.349206349205            |46.06349206349206|1.0158730158730158           |null                    |5.9523809523809526     |


In [32]:
df.groupBy('Tipo de contagio').sum().show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------------+---------------------------------+------------------------------+---------+-----------------------------+------------------------+-----------------------+
|Tipo de contagio|sum(ID de caso)|sum(Código DIVIPOLA departamento)|sum(Código DIVIPOLA municipio)|sum(Edad)|sum(Unidad de medida de edad)|sum(Código ISO del país)|sum(Pertenencia étnica)|
+----------------+---------------+---------------------------------+------------------------------+---------+-----------------------------+------------------------+-----------------------+
|Importado       |203094         |347819                           |14166286                      |20248    |477                          |305340                  |2812                   |
|Comunitaria     |48943          |40146                            |1188202                       |2902     |64                           |null                    |375                    |
|En estudio      |677            |5                    

In [33]:
# Value counts
df.groupBy('Tipo de contagio').max().show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------------+---------------------------------+------------------------------+---------+-----------------------------+------------------------+-----------------------+
|Tipo de contagio|max(ID de caso)|max(Código DIVIPOLA departamento)|max(Código DIVIPOLA municipio)|max(Edad)|max(Unidad de medida de edad)|max(Código ISO del país)|max(Pertenencia étnica)|
+----------------+---------------+---------------------------------+------------------------------+---------+-----------------------------+------------------------+-----------------------+
|Importado       |994            |47001                            |76892                         |87       |1                            |862                     |6                      |
|Comunitaria     |1000           |13001                            |76001                         |82       |2                            |null                    |6                      |
|En estudio      |677            |5                    

In [34]:
# Value counts
df.groupBy('Tipo de contagio').min().show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------------+---------------------------------+------------------------------+---------+-----------------------------+------------------------+-----------------------+
|Tipo de contagio|min(ID de caso)|min(Código DIVIPOLA departamento)|min(Código DIVIPOLA municipio)|min(Edad)|min(Unidad de medida de edad)|min(Código ISO del país)|min(Pertenencia étnica)|
+----------------+---------------+---------------------------------+------------------------------+---------+-----------------------------+------------------------+-----------------------+
|Importado       |1              |5                                |5001                          |9        |1                            |32                      |1                      |
|Comunitaria     |250            |5                                |5001                          |6        |1                            |null                    |5                      |
|En estudio      |677            |5                    

In [35]:
#Aggregation
df.groupBy('Tipo de contagio').agg({'ID de caso':'sum'}).show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------------+
|Tipo de contagio|sum(ID de caso)|
+----------------+---------------+
|Importado       |203094         |
|Comunitaria     |48943          |
|En estudio      |677            |
|Relacionado     |247786         |
+----------------+---------------+

In [36]:
# UDF
from pyspark.sql.functions import udf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
#normal function
def price_range(brand):
    if brand in ['Comunitaria', 'Relacionado']:
        return 'Comunitaria o Relacionado'
    elif brand =='Importado':
        return 'Importado'
    else:
        return 'En estudio'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
#create udf using python function
brand_udf=udf(price_range,StringType())
#apply udf on dataframe
df.withColumn('price_range',brand_udf(df['Tipo de contagio'])).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+-------------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+-------------------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país          |Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|price_range              |
+-----------------+----------+---------------------+----------------------------+-----

In [39]:
#using lambda function
age_udf = udf(lambda Edad: "young" if Edad <= 30 else "senior", StringType())
#apply udf on dataframe
df.withColumn("age_group", age_udf(df.Edad)).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+-------------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+---------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país          |Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|age_group|
+-----------------+----------+---------------------+----------------------------+-------------------+-----------------

In [49]:
#pandas udf
from pyspark.sql.functions import pandas_udf, PandasUDFType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
#create python function
def remaining_yrs(age):
    yrs_left=100-age

    return yrs_left

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
#create udf using python function
length_udf = udf(remaining_yrs, IntegerType())
#apply pandas udf on dataframe
df.withColumn("yrs_left", length_udf(df['Edad'])).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+-------------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+--------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país          |Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|yrs_left|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------

In [53]:
#udf using two columns
def prod(rating,exp):
    x=rating*exp
    return x

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
#create udf using python function
prod_udf = udf(prod, DoubleType())
#apply pandas udf on multiple columns of dataframe
df.withColumn("product", prod_udf(df['Edad'],df['ID de caso'])).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+-------------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+-------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país          |Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|product|
+-----------------+----------+---------------------+----------------------------+-------------------+---------------------

In [55]:
#duplicate values
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1000

In [56]:
#drop duplicate values
df=df.dropDuplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
#validate new count
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1000

In [61]:
#drop column of dataframe
df_new=df.drop('Tipo de contagio')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
df_new.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+------------------+------+-------------------+--------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Ubicación del caso|Estado|Código ISO del país|     Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+---

In [71]:
#target directory
write_uri='s3://jupyterjorge/trusted/datasets/covid19/results'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
#save the dataframe as single csv
df.coalesce(1).write.format("csv").option("header","true").save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
#target location
parquet_uri='s3://jupyterjorge/trusted/datasets/covid19/results_parquet'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
#save the data into parquet format
df.write.format('parquet').save(parquet_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…